In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd

torch.manual_seed(42)

In [1]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

In [7]:
word_to_idx = {}
i = 0

In [8]:
for words, b in data + test_data:
    for word in words:
        if word not in word_to_idx:
            word_to_idx[word] = i
            i += 1

In [22]:
s = [(k, word_to_idx[k]) for k in sorted(word_to_idx, key=word_to_idx.get, reverse=True)]

[('on', 25),
 ('si', 24),
 ('Yo', 23),
 ('at', 22),
 ('lost', 21),
 ('get', 20),
 ('good', 19),
 ('a', 18),
 ('not', 17),
 ('is', 16),
 ('idea', 15),
 ('buena', 14),
 ('una', 13),
 ('sea', 12),
 ('que', 11),
 ('creo', 10),
 ('No', 9),
 ('to', 8),
 ('it', 7),
 ('Give', 6),
 ('cafeteria', 5),
 ('la', 4),
 ('en', 3),
 ('comer', 2),
 ('gusta', 1),
 ('me', 0)]

In [10]:
VOCAB_SIZE = len(word_to_idx)
NUM_LABELS = 2 # English, Spanish

In [41]:
class BOWClassifier(nn.Module):
    
    def __init__(self, num_labels, vocab_size):
        super().__init__()
        
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec))

In [42]:
def make_bow_vector(sentence, word_to_idx):
    vec = torch.zeros(len(word_to_idx))
    for word in sentence:
        vec[word_to_idx[word]] += 1
    return vec.view(1, -1)

In [43]:
make_bow_vector("me gusta comer en la cafeteria".split(), word_to_idx)



Columns 0 to 12 
    1     1     1     1     1     1     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0
[torch.FloatTensor of size 1x26]

In [44]:
def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [45]:
model = BOWClassifier(NUM_LABELS, VOCAB_SIZE)

In [46]:
for param in model.parameters():
    print(param)

Parameter containing:

Columns 0 to 9 
 0.1499  0.1628 -0.0459  0.1802 -0.0430  0.0396 -0.0955  0.1152  0.1729 -0.1439
-0.0554 -0.1179  0.0185 -0.1937  0.1771 -0.1666  0.1514  0.0326 -0.0637  0.1212

Columns 10 to 19 
 0.1705  0.0367  0.1449  0.0266  0.0946 -0.0277  0.1512  0.0290 -0.0916  0.0500
 0.0306  0.1585  0.0214 -0.0618  0.0527 -0.0532  0.0825  0.1751  0.1134 -0.0857

Columns 20 to 25 
-0.0904 -0.0230 -0.0797  0.1301 -0.1548 -0.0904
 0.1132  0.0351  0.0996 -0.1195 -0.1941 -0.0758
[torch.FloatTensor of size 2x26]

Parameter containing:
-0.1504
 0.1609
[torch.FloatTensor of size 2]



In [47]:
model

BOWClassifier(
  (linear): Linear(in_features=26, out_features=2)
)

In [48]:
sample = data[0]
sample

(['me', 'gusta', 'comer', 'en', 'la', 'cafeteria'], 'SPANISH')

In [49]:
bow_vector = make_bow_vector(sample[0], word_to_idx)

In [50]:
log_prob = model(autograd.Variable(bow_vector))

/home/as/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


In [51]:
print(log_prob)

Variable containing:
-0.4855 -0.9556
[torch.FloatTensor of size 1x2]



In [52]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [54]:
for instance, label in test_data:
    bow_vec = make_bow_vector(instance, word_to_idx)
    bow_vec = autograd.Variable(bow_vec)
    log_prob = model(bow_vec)
    print(log_prob)

Variable containing:
-0.6953 -0.6910
[torch.FloatTensor of size 1x2]

Variable containing:
-0.7070 -0.6795
[torch.FloatTensor of size 1x2]



/home/as/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


In [56]:
print(next(model.parameters())[:, word_to_idx["creo"]])

Variable containing:
 0.1705
 0.0306
[torch.FloatTensor of size 2]



In [57]:
loss_fn = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [59]:
for epoch in range(1000):
    for instance, label in data:
        model.zero_grad()
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_idx))
        target = autograd.Variable(make_target(label, label_to_ix))
        
        log_prob = model(bow_vec)
        
        loss = loss_fn(log_prob, target)
        loss.backward()
        optimizer.step()

/home/as/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


In [60]:
for instance, label in test_data:
    bow_vec = make_bow_vector(instance, word_to_idx)
    bow_vec = autograd.Variable(bow_vec)
    log_prob = model(bow_vec)
    print(log_prob)

Variable containing:
-0.0583 -2.8706
[torch.FloatTensor of size 1x2]

Variable containing:
-3.8550 -0.0214
[torch.FloatTensor of size 1x2]



/home/as/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


In [61]:
print(next(model.parameters())[:, word_to_idx["creo"]])

Variable containing:
 0.8141
-0.6130
[torch.FloatTensor of size 2]



In [68]:
t = next(model.parameters())

In [72]:
t[:, 19]

Variable containing:
-0.2725
 0.2368
[torch.FloatTensor of size 2]